### Libs 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, KFold
import sys
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tabulate import tabulate
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier


### Dados

#### features extraction

In [2]:
FEATURES_SET = {
    "feature": 1,
    "permission": 2,
    "activity": 3,
    "service_receiver": 3,
    "provider": 3,
    "service": 3,
    "intent": 4,
    "api_call": 5,
    "real_permission": 6,
    "call": 7,
    "url": 8
}


def count_feature_set(lines):
    """
    Count how many features belong to a specific set
    :param lines: features in the text file
    :return:
    """
    features_map = {x: 0 for x in range(1, 9)}
    for l in lines:
        if l != "\n":
            set = l.split("::")[0]
            features_map[FEATURES_SET[set]] += 1
    features = []
    for i in range(1, 9):
        features.append(features_map[i])
    return features


In [3]:
def read(LOAD_DATA=False):
    if LOAD_DATA:
        print("Previous data not loaded. Attempt to read data ...")
        mypath = r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton\Drebin\MetaData\feature_vectors\feature_vectors"
        onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

        print("Reading csv file for ground truth ...")
        ground_truth = np.loadtxt(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton\Drebin\MetaData\sha256_family.csv", delimiter=",", skiprows=1, dtype=str)
        # print ground_truth.shape
        # families = np.unique(ground_truth[:, 1])
        # print families
        # print len(families)

        print("Reading positive and negative texts ...")
        pos = []
        neg = []
        for virus in tqdm(onlyfiles):
            if virus in ground_truth[:, 0]:
                pos.append(virus)
            else:
                #if len(neg) < 5560:
                #if len(neg) < 22240:
                    neg.append(virus)

        print("Extracting features ...")
        x = []
        y = []
        for text_file in tqdm(pos):
            sys.stdin = open("%s/%s" % (mypath, text_file))
            features = sys.stdin.readlines()
            sample = count_feature_set(features)
            x.append(sample)
            y.append(1)

        for text_file in tqdm(neg):
            sys.stdin = open("%s/%s" % (mypath, text_file))
            features = sys.stdin.readlines()
            sample = count_feature_set(features)
            x.append(sample)
            y.append(0)

        print("Data is read successfully:")
        x = np.array(x)
        y = np.array(y)
        print(x.shape, y.shape)

        print("Saving data under data_numpy directory ...")
        np.save(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy", x)
        np.save(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy", y)

        return x, y
    else:
        print("Loading previous data ...")
        x_ = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy")
        y_ = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy")
        print(x_.shape, y_.shape)
        # print x == x_, y == y_
        return x_, y_


def map_family_to_category(families):
    out = {}
    count = 1
    for family in families:
        out[family] = count
        count += 1
    return out


if __name__ == "__main__":
    x, y = read(LOAD_DATA=True)

Previous data not loaded. Attempt to read data ...
Reading csv file for ground truth ...
Reading positive and negative texts ...


100%|██████████| 129013/129013 [00:01<00:00, 69773.39it/s]


Extracting features ...


100%|██████████| 123453/123453 [10:18<00:00, 199.55it/s]

Data is read successfully:
(129013, 8) (129013,)
Saving data under data_numpy directory ...


In [4]:
x_all, y_all = read(LOAD_DATA=False)
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

Loading previous data ...
(129013, 8) (129013,)
(103210, 8) (103210,)
(25803, 8) (25803,)


#### initialization of results vectors

In [5]:
# # Contagem de classes antes da divisão
# y_all_counts = pd.Series(y_all).value_counts(normalize=True)

# # Divisão dos dados em treinamento e teste
# x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42, stratify=y_all)

# # Contagem de classes após a divisão
# y_train_counts = pd.Series(y_train).value_counts(normalize=True)
# y_test_counts = pd.Series(y_test).value_counts(normalize=True)

# # Verificação se as proporções foram mantidas
# print("Proporções das classes no conjunto original:")
# print(y_all_counts)
# print("\nProporções das classes no conjunto de treinamento:")
# print(y_train_counts)
# print("\nProporções das classes no conjunto de teste:")
# print(y_test_counts)

In [6]:
####### DP = Desvio Padrão; PosBal = Pos Balanceamento
Acurracy = []
Precision = []
Recall = []
F1_score = []
Roc_auc = []
######
AcurracyDP = []
PrecisionDP = []
RecallDP = []
F1_scoreDP = []
Roc_aucDP = []
#########################################
AcurracyBagging = []
PrecisionBagging = []
RecallBagging = []
F1Bagging = []
Roc_aucBagging = []
######
AcurracyBaggingDP = []
PrecisionBaggingDP = []
RecallBaggingDP = []
F1BaggingDP = []
Roc_aucBaggingDP = []
########################################
AcurracyEnsemble = []
PrecisionEnsemble = []
RecallEnsemble = []
F1Ensemble = []
Roc_aucEnsemble = []
######
AcurracyEnsembleDP = []
PrecisionEnsembleDP = []
RecallEnsembleDP = []
F1EnsembleDP = []
Roc_aucEnsembleDP = []
###########################################
AcurracyPosBal = []
PrecisionPosBal = []
RecallPosBal = []
F1_scorePosBal = []
Roc_aucPosBal = []
######
AcurracyDPPosBal = []
PrecisionDPPosBal = []
RecallDPPosBal = []
F1_scoreDPPosBal = []
Roc_aucDPPosBal = []
######
AcurracyBaggingPosBal = []
PrecisionBaggingPosBal = []
RecallBaggingPosBal = []
F1BaggingPosBal = []
Roc_aucBaggingPosBal = []
######
AcurracyBaggingDPPosBal = []
PrecisionBaggingDPPosBal = []
RecallBaggingDPPosBal = []
F1BaggingDPPosBal = []
Roc_aucBaggingDPPosBal = []

### Modelos Base

#### Decision Tree (DT1)

In [7]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)

    # Inicializar o modelo de árvore de decisão com Gini Impurity e sem limitação de altura
    dt1 = DecisionTreeClassifier(criterion='gini')

    # Treinar o modelo no conjunto de treino
    dt1.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = dt1.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:00<00:07,  3.99it/s]

Repetição 1 - Acurácia: 0.98, Precisão: 0.74, Recall: 0.83, F1-score: 0.78, ROC AUC: 0.91


  7%|▋         | 2/30 [00:00<00:07,  4.00it/s]

Repetição 2 - Acurácia: 0.98, Precisão: 0.77, Recall: 0.85, F1-score: 0.81, ROC AUC: 0.92


 10%|█         | 3/30 [00:00<00:06,  4.03it/s]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.78, Recall: 0.81, F1-score: 0.80, ROC AUC: 0.90


 97%|█████████▋| 29/30 [00:07<00:00,  4.05it/s]

Repetição 29 - Acurácia: 0.98, Precisão: 0.79, Recall: 0.81, F1-score: 0.80, ROC AUC: 0.90


100%|██████████| 30/30 [00:07<00:00,  4.02it/s]

Repetição 30 - Acurácia: 0.98, Precisão: 0.76, Recall: 0.83, F1-score: 0.80, ROC AUC: 0.91
Média das Acurácias: 0.98165 ± 0.00075
Média das Precisões: 0.76891 ± 0.01246
Média dos Recalls: 0.82155 ± 0.01335
Média dos F1-scores: 0.79423 ± 0.00781
Média dos ROC AUC: 0.90521 ± 0.00649


#### Naive Bayes (NB1)

In [8]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
    
    #Inicializar o modelo de Naive Bayes com distribuição de Bernoulli
    nb1 = GaussianNB()

    # Treinar o modelo no conjunto de treino
    nb1.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = nb1.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  7%|▋         | 2/30 [00:00<00:01, 14.40it/s]

Repetição 1 - Acurácia: 0.92, Precisão: 0.26, Recall: 0.46, F1-score: 0.33, ROC AUC: 0.70
Repetição 2 - Acurácia: 0.92, Precisão: 0.25, Recall: 0.47, F1-score: 0.33, ROC AUC: 0.70
Repetição 3 - Acurácia: 0.92, Precisão: 0.25, Recall: 0.46, F1-score: 0.32, ROC AUC: 0.70


 13%|█▎        | 4/30 [00:00<00:01, 13.46it/s]

Repetição 4 - Acurácia: 0.92, Precisão: 0.25, Recall: 0.46, F1-score: 0.33, ROC AUC: 0.70
Repetição 5 - Acurácia: 0.92, Precisão: 0.24, Recall: 0.45, F1-score: 0.31, ROC AUC: 0.69


 20%|██        | 6/30 [00:00<00:01, 13.89it/s]

...

Repetição 22 - Acurácia: 0.92, Precisão: 0.25, Recall: 0.46, F1-score: 0.33, ROC AUC: 0.70
Repetição 23 - Acurácia: 0.92, Precisão: 0.28, Recall: 0.47, F1-score: 0.35, ROC AUC: 0.71
Repetição 24 - Acurácia: 0.92, Precisão: 0.25, Recall: 0.46, F1-score: 0.32, ROC AUC: 0.70
Repetição 25 - Acurácia: 0.92, Precisão: 0.24, Recall: 0.44, F1-score: 0.31, ROC AUC: 0.69


 93%|█████████▎| 28/30 [00:01<00:00, 14.37it/s]

Repetição 26 - Acurácia: 0.92, Precisão: 0.27, Recall: 0.45, F1-score: 0.33, ROC AUC: 0.70
Repetição 27 - Acurácia: 0.92, Precisão: 0.25, Recall: 0.47, F1-score: 0.33, ROC AUC: 0.70
Repetição 28 - Acurácia: 0.92, Precisão: 0.25, Recall: 0.44, F1-score: 0.32, ROC AUC: 0.69


100%|██████████| 30/30 [00:02<00:00, 14.22it/s]

Repetição 29 - Acurácia: 0.92, Precisão: 0.24, Recall: 0.44, F1-score: 0.31, ROC AUC: 0.69
Repetição 30 - Acurácia: 0.92, Precisão: 0.26, Recall: 0.47, F1-score: 0.33, ROC AUC: 0.70
Média das Acurácias: 0.91850 ± 0.00267
Média das Precisões: 0.25439 ± 0.00978
Média dos Recalls: 0.46055 ± 0.01427
Média dos F1-scores: 0.32764 ± 0.01015
Média dos ROC AUC: 0.69984 ± 0.00701


####  MLP1

In [9]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
    
    # Inicializar o modelo MLP
    mlp1 = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

    # Treinar o modelo no conjunto de treino
    mlp1.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = mlp1.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:42<20:29, 42.41s/it]

Repetição 1 - Acurácia: 0.98, Precisão: 0.84, Recall: 0.59, F1-score: 0.69, ROC AUC: 0.79


  7%|▋         | 2/30 [01:14<17:00, 36.45s/it]

Repetição 2 - Acurácia: 0.98, Precisão: 0.88, Recall: 0.49, F1-score: 0.63, ROC AUC: 0.75


 10%|█         | 3/30 [01:56<17:27, 38.79s/it]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.79, Recall: 0.66, F1-score: 0.72, ROC AUC: 0.83


 97%|█████████▋| 29/30 [18:00<00:35, 35.03s/it]

Repetição 29 - Acurácia: 0.98, Precisão: 0.82, Recall: 0.57, F1-score: 0.67, ROC AUC: 0.78


100%|██████████| 30/30 [18:45<00:00, 37.50s/it]

Repetição 30 - Acurácia: 0.98, Precisão: 0.81, Recall: 0.63, F1-score: 0.71, ROC AUC: 0.81
Média das Acurácias: 0.97673 ± 0.00113
Média das Precisões: 0.81462 ± 0.04816
Média dos Recalls: 0.60309 ± 0.04891
Média dos F1-scores: 0.68991 ± 0.02078
Média dos ROC AUC: 0.79832 ± 0.02333


#### KNN1

In [10]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)
    
    #inicializar o modelo KNN
    knn1 = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

    # Treinar o modelo no conjunto de treino
    knn1.fit(np.array(x_train), y_train)

    # Fazer previsões no conjunto de teste
    previsoes = knn1.predict(np.array(x_test))

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
Acurracy.append(media_acuracias)
Precision.append(media_precisoes)
Recall.append(media_recalls)
F1_score.append(media_f1_scores)
Roc_auc.append(media_roc_auc_scores)

AcurracyDP.append(dp_acuracias)
PrecisionDP.append(dp_precisoes)
RecallDP.append(dp_recalls)
F1_scoreDP.append(dp_f1_scores)
Roc_aucDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:01<00:49,  1.70s/it]

Repetição 1 - Acurácia: 0.98, Precisão: 0.80, Recall: 0.73, F1-score: 0.77, ROC AUC: 0.86


  7%|▋         | 2/30 [00:03<00:47,  1.69s/it]

Repetição 2 - Acurácia: 0.98, Precisão: 0.83, Recall: 0.73, F1-score: 0.78, ROC AUC: 0.86


 10%|█         | 3/30 [00:05<00:46,  1.71s/it]

...

Repetição 28 - Acurácia: 0.98, Precisão: 0.79, Recall: 0.73, F1-score: 0.76, ROC AUC: 0.86


 97%|█████████▋| 29/30 [00:49<00:01,  1.75s/it]

Repetição 29 - Acurácia: 0.98, Precisão: 0.82, Recall: 0.74, F1-score: 0.78, ROC AUC: 0.87


100%|██████████| 30/30 [00:51<00:00,  1.71s/it]

Repetição 30 - Acurácia: 0.98, Precisão: 0.80, Recall: 0.74, F1-score: 0.77, ROC AUC: 0.87
Média das Acurácias: 0.98109 ± 0.00066
Média das Precisões: 0.81231 ± 0.01052
Média dos Recalls: 0.73004 ± 0.01214
Média dos F1-scores: 0.76890 ± 0.00839
Média dos ROC AUC: 0.86122 ± 0.00602


#### Resultados

In [11]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Acurracy, AcurracyDP)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Precision, PrecisionDP)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Recall, RecallDP)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_score, F1_scoreDP)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_auc, Roc_aucDP)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9817 ± 0.0008,0.7689 ± 0.0125,0.8216 ± 0.0133,0.7942 ± 0.0078,0.9052 ± 0.0065
1,Naive Bayes,0.9185 ± 0.0027,0.2544 ± 0.0098,0.4606 ± 0.0143,0.3276 ± 0.0102,0.6998 ± 0.0070
2,MLP,0.9767 ± 0.0011,0.8146 ± 0.0482,0.6031 ± 0.0489,0.6899 ± 0.0208,0.7983 ± 0.0233
3,KNN,0.9811 ± 0.0007,0.8123 ± 0.0105,0.7300 ± 0.0121,0.7689 ± 0.0084,0.8612 ± 0.0060


### Bagging

#### Decision Tree (DT2)

In [12]:
# Definir o classificador de Bagging com Decision Tree como base
dt2 = BaggingClassifier(estimator = dt1, n_estimators=30)

# Treinar o modelo Bagging
dt2.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = dt2.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Decision Tree:", accuracy_bagging)
print("Precisão do modelo Bagging com Decision Tree:", precision_bagging)
print("Recall do modelo Bagging com Decision Tree:", recall_bagging)
print("F1-score do modelo Bagging com Decision Tree:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Decision Tree:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Decision Tree:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com Decision Tree: 0.9872882998100996
Precisão do modelo Bagging com Decision Tree: 0.8967611336032388
Recall do modelo Bagging com Decision Tree: 0.7967625899280576
F1-score do modelo Bagging com Decision Tree: 0.8438095238095238
AUC ROC do modelo Bagging com Decision Tree: 0.8963157650138446
Matriz de confusão do modelo Bagging com Decision Tree:
[[24589   102]
 [  226   886]]


#### Naive Bayes (NB2)

In [13]:
# Definir o classificador de Bagging com Naive Bayes como base
nb2 = BaggingClassifier(estimator=nb1, n_estimators=30)

# Treinar o modelo Bagging
nb2.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = nb2.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Naive Bayes:", accuracy_bagging)
print("Precisão do modelo Bagging com Naive Bayes:", precision_bagging)
print("Recall do modelo Bagging com Naive Bayes:", recall_bagging)
print("F1-score do modelo Bagging com Naive Bayes:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Naive Bayes:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Naive Bayes:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com Naive Bayes: 0.9163663139944968
Precisão do modelo Bagging com Naive Bayes: 0.2509523809523809
Recall do modelo Bagging com Naive Bayes: 0.4739208633093525
F1-score do modelo Bagging com Naive Bayes: 0.32814445828144456
AUC ROC do modelo Bagging com Naive Bayes: 0.705106719775854
Matriz de confusão do modelo Bagging com Naive Bayes:
[[23118  1573]
 [  585   527]]


#### MLP2

In [14]:
# Definir o classificador de Bagging com MLP como base
mlp2 = BaggingClassifier(estimator=mlp1, n_estimators=30)

# Treinar o modelo Bagging
mlp2.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = mlp2.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com MLP:", accuracy_bagging)
print("Precisão do modelo Bagging com MLP:", precision_bagging)
print("Recall do modelo Bagging com MLP:", recall_bagging)
print("F1-score do modelo Bagging com MLP:", f1_score_bagging)
print("AUC ROC do modelo Bagging com MLP:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com MLP:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com MLP: 0.9781033213192264
Precisão do modelo Bagging com MLP: 0.8397515527950311
Recall do modelo Bagging com MLP: 0.6079136690647482
F1-score do modelo Bagging com MLP: 0.7052686489306208
AUC ROC do modelo Bagging com MLP: 0.8013445466541999
Matriz de confusão do modelo Bagging com MLP:
[[24562   129]
 [  436   676]]


#### KNN2

In [15]:
# Definir o classificador de Bagging com KNN como base
knn2 = BaggingClassifier(estimator=knn1, n_estimators=30)

# Treinar o modelo Bagging
knn2.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = knn2.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com KNN:", accuracy_bagging)
print("Precisão do modelo Bagging com KNN:", precision_bagging)
print("Recall do modelo Bagging com KNN:", recall_bagging)
print("F1-score do modelo Bagging com KNN:", f1_score_bagging)
print("AUC ROC do modelo Bagging com KNN:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com KNN:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBagging.append(accuracy_bagging)
PrecisionBagging.append(precision_bagging)
RecallBagging.append(recall_bagging)
F1Bagging.append(f1_score_bagging)
Roc_aucBagging.append(auc_roc_bagging)

Acurácia do modelo Bagging com KNN: 0.9816300430182537
Precisão do modelo Bagging com KNN: 0.8241869918699187
Recall do modelo Bagging com KNN: 0.72931654676259
F1-score do modelo Bagging com KNN: 0.773854961832061
AUC ROC do modelo Bagging com KNN: 0.8611549725834334
Matriz de confusão do modelo Bagging com KNN:
[[24518   173]
 [  301   811]]


#### Resultados

In [16]:
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']
# Criar um DataFrame do Pandas com os valores das variáveis
dfB = pd.DataFrame({'Modelos Bagging': Modelos, 'Acuracia (DREBIN)': AcurracyBagging,
                    'Precisão (DREBIN)': PrecisionBagging, 'Recall (DREBIN)':RecallBagging, 
                    'F1 Score': F1Bagging, 'AUC ROC': Roc_aucBagging})

print(tabulate(dfB, headers='keys', tablefmt='fancy_grid', floatfmt=".4f"))

╒════╤═══════════════════╤═════════════════════╤═════════════════════╤═══════════════════╤════════════╤═══════════╕
│    │ Modelos Bagging   │   Acuracia (DREBIN) │   Precisão (DREBIN) │   Recall (DREBIN) │   F1 Score │   AUC ROC │
╞════╪═══════════════════╪═════════════════════╪═════════════════════╪═══════════════════╪════════════╪═══════════╡
│  0 │ Decision Tree     │              0.9873 │              0.8968 │            0.7968 │     0.8438 │    0.8963 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  1 │ Naive Bayes       │              0.9164 │              0.2510 │            0.4739 │     0.3281 │    0.7051 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  2 │ MLP               │              0.9781 │              0.8398 │            0.6079 │     0.7053 │    0.8013 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼──

### Ensemble

#### Random Forest

In [17]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)

    rf = RandomForestClassifier(n_estimators=100)

    rf.fit(x_train, y_train)
    
    # Fazer previsões no conjunto de teste
    previsoes = rf.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyEnsemble.append(media_acuracias)
PrecisionEnsemble.append(media_precisoes)
RecallEnsemble.append(media_recalls)
F1Ensemble.append(media_f1_scores)
Roc_aucEnsemble.append(media_roc_auc_scores)

AcurracyEnsembleDP.append(dp_acuracias)
PrecisionEnsembleDP.append(dp_precisoes)
RecallEnsembleDP.append(dp_recalls)
F1EnsembleDP.append(dp_f1_scores)
Roc_aucEnsembleDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:04<02:14,  4.64s/it]

Repetição 1 - Acurácia: 0.99, Precisão: 0.92, Recall: 0.80, F1-score: 0.85, ROC AUC: 0.90


  7%|▋         | 2/30 [00:09<02:09,  4.64s/it]

Repetição 2 - Acurácia: 0.99, Precisão: 0.95, Recall: 0.81, F1-score: 0.88, ROC AUC: 0.90


 10%|█         | 3/30 [00:13<02:05,  4.66s/it]

...

Repetição 28 - Acurácia: 0.99, Precisão: 0.93, Recall: 0.80, F1-score: 0.86, ROC AUC: 0.90


 97%|█████████▋| 29/30 [02:14<00:04,  4.63s/it]

Repetição 29 - Acurácia: 0.99, Precisão: 0.93, Recall: 0.81, F1-score: 0.87, ROC AUC: 0.90


100%|██████████| 30/30 [02:19<00:00,  4.64s/it]

Repetição 30 - Acurácia: 0.99, Precisão: 0.91, Recall: 0.83, F1-score: 0.87, ROC AUC: 0.91
Média das Acurácias: 0.98911 ± 0.00060
Média das Precisões: 0.93574 ± 0.00898
Média dos Recalls: 0.80258 ± 0.01349
Média dos F1-scores: 0.86397 ± 0.00811
Média dos ROC AUC: 0.90005 ± 0.00670


#### Gradient Boosted Decision Tree

In [18]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all)

    gb = GradientBoostingClassifier(n_estimators=100)

    gb.fit(x_train, y_train)
    
    # Fazer previsões no conjunto de teste
    previsoes = gb.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyEnsemble.append(media_acuracias)
PrecisionEnsemble.append(media_precisoes)
RecallEnsemble.append(media_recalls)
F1Ensemble.append(media_f1_scores)
Roc_aucEnsemble.append(media_roc_auc_scores)

AcurracyEnsembleDP.append(dp_acuracias)
PrecisionEnsembleDP.append(dp_precisoes)
RecallEnsembleDP.append(dp_recalls)
F1EnsembleDP.append(dp_f1_scores)
Roc_aucEnsembleDP.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:05<02:37,  5.43s/it]

Repetição 1 - Acurácia: 0.97, Precisão: 0.84, Recall: 0.45, F1-score: 0.59, ROC AUC: 0.73


  7%|▋         | 2/30 [00:10<02:31,  5.42s/it]

Repetição 2 - Acurácia: 0.97, Precisão: 0.84, Recall: 0.45, F1-score: 0.58, ROC AUC: 0.72


 10%|█         | 3/30 [00:16<02:25,  5.38s/it]

...

Repetição 28 - Acurácia: 0.97, Precisão: 0.84, Recall: 0.48, F1-score: 0.61, ROC AUC: 0.74


 97%|█████████▋| 29/30 [02:35<00:05,  5.33s/it]

Repetição 29 - Acurácia: 0.97, Precisão: 0.84, Recall: 0.46, F1-score: 0.59, ROC AUC: 0.73


100%|██████████| 30/30 [02:40<00:00,  5.34s/it]

Repetição 30 - Acurácia: 0.97, Precisão: 0.87, Recall: 0.48, F1-score: 0.62, ROC AUC: 0.74
Média das Acurácias: 0.97327 ± 0.00102
Média das Precisões: 0.83838 ± 0.01827
Média dos Recalls: 0.47062 ± 0.01937
Média dos F1-scores: 0.60265 ± 0.01851
Média dos ROC AUC: 0.73327 ± 0.00972


#### Classificadores

In [19]:
classifiers = {
    'MLP': mlp1,
    'Random Forest': rf,
    'Decision Tree': dt1,
    'Gradient Boosted Decision Tree':gb,
    'Naive Bayes': nb1,
    'KNN': knn1,
    'MLP Bagging': mlp2,
    'Naive Bayes Bagging': nb2,
    'KNN Bagging': knn2,
    'Decision Tree Bagging': dt2
}

#### Overall Local Accuracy (OLA1)

In [20]:
# Lista para armazenar os classificadores treinados
trained_classifiers = {name: [] for name in classifiers.keys()}

# Lista para armazenar as precisões locais
local_accuracies = {name: [] for name in classifiers.keys()}

# Validacao cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_index, val_index in kf.split(x_train):
    X_train_fold, X_val_fold = x_train[train_index], x_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    for name, clf in classifiers.items():
        # Treinar o classificador
        clf.fit(X_train_fold, y_train_fold)
        # Fazer previsões
        y_pred = clf.predict(X_val_fold)
        # Calcular a acurácia local
        accuracy = accuracy_score(y_val_fold, y_pred)
        # Armazenar o classificador treinado e a precisão local
        trained_classifiers[name].append(clf)
        local_accuracies[name].append(accuracy)

In [21]:
# Função para prever usando OLA
def predict_ola(X_test):
    predictions = []
    for sample in X_test:
        best_clf_name = None
        best_accuracy = -1
        best_prediction = None
        
        for name in classifiers.keys():
            # Obter a média das precisões locais para o classificador
            avg_accuracy = np.mean(local_accuracies[name])
            # Usar o classificador com a maior precisão local
            if avg_accuracy > best_accuracy:
                best_accuracy = avg_accuracy
                best_clf_name = name
        
        # Fazer a previsão com o melhor classificador
        best_clf = trained_classifiers[best_clf_name][0]
        prediction = best_clf.predict([sample])[0]
        predictions.append(prediction)
    
    return predictions

# Fazer previsões no conjunto de teste usando OLA
y_pred_ola = predict_ola(x_test)

# Avaliar a acurácia
accuracy_ola = accuracy_score(y_test, y_pred_ola)



In [25]:
print(accuracy_ola)

0.9889935278843546


#### K-Nearest Oracle Elimination (KNORA-E)

#### K-Nearest Oracle Union (KNORA-U)

#### K-Nearest Oracle Profile (KNOP)

#### META-DES

### Balanceamento de dados

In [22]:
smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=5)
x_train, y_train = smote.fit_resample(x_train, y_train)


print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(197524, 8) (197524,)
(25803, 8) (25803,)


In [23]:
# Calcular proporção das classes em y_all
unique_classes, class_counts = np.unique(y_test, return_counts=True)
proporcao_classes_resampled = class_counts / len(y_test)

print("Proporção das classes no conjunto teste:")
for classe, proporcao in zip(unique_classes, proporcao_classes_resampled):
    print(f"Classe {classe}: {proporcao:.2f}")

# Calcular proporção das classes em y_resampled
unique_classes, class_counts = np.unique(y_train, return_counts=True)
proporcao_classes_resampled = class_counts / len(y_train)

print("Proporção das classes no conjunto treino:")
for classe, proporcao in zip(unique_classes, proporcao_classes_resampled):
    print(f"Classe {classe}: {proporcao:.2f}")

Proporção das classes no conjunto teste:
Classe 0: 0.96
Classe 1: 0.04
Proporção das classes no conjunto treino:
Classe 0: 0.50
Classe 1: 0.50


### Modelos Base após o Balanceamento

#### Decision Tree

In [24]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):
    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    # Inicializar o modelo de árvore de decisão com Gini Impurity e sem limitação de altura
    modelo_arvore = DecisionTreeClassifier(criterion='gini')

    # Treinar o modelo no conjunto de treino
    modelo_arvore.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_arvore.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  0%|          | 0/30 [00:00<?, ?it/s]


NameError: name 'X_resampled' is not defined

#### Naive Bayes

In [ ]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    
    #Inicializar o modelo de Naive Bayes com distribuição de Bernoulli
    modelo_naive_bayes = GaussianNB()

    # Treinar o modelo no conjunto de treino
    modelo_naive_bayes.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_naive_bayes.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:00<00:04,  6.53it/s]

Repetição 1 - Acurácia: 0.73, Precisão: 0.73, Recall: 0.74, F1-score: 0.73, ROC AUC: 0.73


  7%|▋         | 2/30 [00:00<00:03,  7.06it/s]

Repetição 2 - Acurácia: 0.73, Precisão: 0.73, Recall: 0.74, F1-score: 0.73, ROC AUC: 0.73


 10%|█         | 3/30 [00:00<00:03,  7.27it/s]

...

Repetição 28 - Acurácia: 0.73, Precisão: 0.72, Recall: 0.74, F1-score: 0.73, ROC AUC: 0.73


 97%|█████████▋| 29/30 [00:03<00:00,  7.38it/s]

Repetição 29 - Acurácia: 0.74, Precisão: 0.74, Recall: 0.74, F1-score: 0.74, ROC AUC: 0.74


100%|██████████| 30/30 [00:04<00:00,  7.43it/s]

Repetição 30 - Acurácia: 0.73, Precisão: 0.73, Recall: 0.74, F1-score: 0.73, ROC AUC: 0.73
Média das Acurácias: 0.72989 ± 0.00460
Média das Precisões: 0.72553 ± 0.00473
Média dos Recalls: 0.73969 ± 0.01407
Média dos F1-scores: 0.73245 ± 0.00672
Média dos ROC AUC: 0.72989 ± 0.00460


#### MLP

In [ ]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    
    # Inicializar o modelo MLP
    modelo_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

    # Treinar o modelo no conjunto de treino
    modelo_mlp.fit(x_train, y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_mlp.predict(x_test)

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [01:43<49:47, 103.03s/it]

Repetição 1 - Acurácia: 0.93, Precisão: 0.92, Recall: 0.95, F1-score: 0.94, ROC AUC: 0.93


  7%|▋         | 2/30 [03:13<44:42, 95.81s/it] 

Repetição 2 - Acurácia: 0.93, Precisão: 0.92, Recall: 0.95, F1-score: 0.94, ROC AUC: 0.93


 10%|█         | 3/30 [04:37<40:37, 90.27s/it]

...

Repetição 28 - Acurácia: 0.94, Precisão: 0.93, Recall: 0.94, F1-score: 0.94, ROC AUC: 0.94


 97%|█████████▋| 29/30 [46:34<01:26, 86.61s/it]

Repetição 29 - Acurácia: 0.94, Precisão: 0.92, Recall: 0.95, F1-score: 0.94, ROC AUC: 0.94


100%|██████████| 30/30 [47:38<00:00, 95.29s/it]

Repetição 30 - Acurácia: 0.93, Precisão: 0.93, Recall: 0.94, F1-score: 0.93, ROC AUC: 0.93
Média das Acurácias: 0.93477 ± 0.00278
Média das Precisões: 0.93293 ± 0.00851
Média dos Recalls: 0.93709 ± 0.01225
Média dos F1-scores: 0.93490 ± 0.00317
Média dos ROC AUC: 0.93477 ± 0.00278


#### KNN

In [ ]:
num_repeticoes = 30

# Listas para armazenar as métricas
acuracias = []
precisoes = []
recalls = []
f1_scores = []
roc_auc_scores = []

# Loop para repetir o processo
for i in tqdm(range(num_repeticoes)):

    x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled)
    
    #inicializar o modelo KNN
    modelo_knn = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

    # Treinar o modelo no conjunto de treino
    modelo_knn.fit(np.array(x_train), y_train)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_knn.predict(np.array(x_test))

    # Avaliar as métricas do modelo e armazenar na lista
    acuracia = accuracy_score(y_test, previsoes)
    precisao = precision_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    roc_auc = roc_auc_score(y_test, previsoes)
    
    acuracias.append(acuracia)
    precisoes.append(precisao)
    recalls.append(recall)
    f1_scores.append(f1)
    roc_auc_scores.append(roc_auc)

    print(f'Repetição {i+1} - Acurácia: {acuracia:.2f}, Precisão: {precisao:.2f}, Recall: {recall:.2f}, F1-score: {f1:.2f}, ROC AUC: {roc_auc:.2f}')

# Calcular as médias das métricas
media_acuracias = np.mean(acuracias)
media_precisoes = np.mean(precisoes)
media_recalls = np.mean(recalls)
media_f1_scores = np.mean(f1_scores)
media_roc_auc_scores = np.mean(roc_auc_scores)

# Calcular os desvios padrão das métricas
dp_acuracias = np.std(acuracias)
dp_precisoes = np.std(precisoes)
dp_recalls = np.std(recalls)
dp_f1_scores = np.std(f1_scores)
dp_roc_auc_scores = np.std(roc_auc_scores)

# Adicionar as médias e desvios padrão às listas de métricas
AcurracyPosBal.append(media_acuracias)
PrecisionPosBal.append(media_precisoes)
RecallPosBal.append(media_recalls)
F1_scorePosBal.append(media_f1_scores)
Roc_aucPosBal.append(media_roc_auc_scores)

AcurracyDPPosBal.append(dp_acuracias)
PrecisionDPPosBal.append(dp_precisoes)
RecallDPPosBal.append(dp_recalls)
F1_scoreDPPosBal.append(dp_f1_scores)
Roc_aucDPPosBal.append(dp_roc_auc_scores)

print(f'Média das Acurácias: {media_acuracias:.5f} ± {dp_acuracias:.5f}')
print(f'Média das Precisões: {media_precisoes:.5f} ± {dp_precisoes:.5f}')
print(f'Média dos Recalls: {media_recalls:.5f} ± {dp_recalls:.5f}')
print(f'Média dos F1-scores: {media_f1_scores:.5f} ± {dp_f1_scores:.5f}')
print(f'Média dos ROC AUC: {media_roc_auc_scores:.5f} ± {dp_roc_auc_scores:.5f}')

  3%|▎         | 1/30 [00:04<02:10,  4.51s/it]

Repetição 1 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97


  7%|▋         | 2/30 [00:08<02:05,  4.48s/it]

Repetição 2 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97


 10%|█         | 3/30 [00:12<01:55,  4.27s/it]

...

Repetição 28 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97


 97%|█████████▋| 29/30 [01:55<00:03,  3.89s/it]

Repetição 29 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97


100%|██████████| 30/30 [01:59<00:00,  3.99s/it]

Repetição 30 - Acurácia: 0.97, Precisão: 0.95, Recall: 0.99, F1-score: 0.97, ROC AUC: 0.97
Média das Acurácias: 0.97081 ± 0.00098
Média das Precisões: 0.95170 ± 0.00165
Média dos Recalls: 0.99197 ± 0.00137
Média dos F1-scores: 0.97142 ± 0.00095
Média dos ROC AUC: 0.97081 ± 0.00098


#### Resultados

In [ ]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(AcurracyPosBal, AcurracyDPPosBal)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(PrecisionPosBal, PrecisionDPPosBal)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(RecallPosBal, RecallDPPosBal)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_scorePosBal, F1_scoreDPPosBal)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_aucPosBal, Roc_aucDPPosBal)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9810 ± 0.0006,0.9737 ± 0.0011,0.9887 ± 0.0008,0.9812 ± 0.0006,0.9810 ± 0.0006
1,Naive Bayes,0.7299 ± 0.0046,0.7255 ± 0.0047,0.7397 ± 0.0141,0.7325 ± 0.0067,0.7299 ± 0.0046
2,MLP,0.9348 ± 0.0028,0.9329 ± 0.0085,0.9371 ± 0.0122,0.9349 ± 0.0032,0.9348 ± 0.0028
3,KNN,0.9708 ± 0.0010,0.9517 ± 0.0017,0.9920 ± 0.0014,0.9714 ± 0.0009,0.9708 ± 0.0010


### Bagging após o Balanceamento

#### Decision Tree

In [ ]:
# Definir o classificador Decision Tree
modelo_arvore = DecisionTreeClassifier(criterion='gini')

# Definir o classificador de Bagging com Decision Tree como base
bagging_clf = BaggingClassifier(estimator = modelo_arvore, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Decision Tree:", accuracy_bagging)
print("Precisão do modelo Bagging com Decision Tree:", precision_bagging)
print("Recall do modelo Bagging com Decision Tree:", recall_bagging)
print("F1-score do modelo Bagging com Decision Tree:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Decision Tree:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Decision Tree:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBaggingPosBal.append(accuracy_bagging)
PrecisionBaggingPosBal.append(precision_bagging)
RecallBaggingPosBal.append(recall_bagging)
F1BaggingPosBal.append(f1_score_bagging)
Roc_aucBaggingPosBal.append(auc_roc_bagging)



Acurácia do modelo Bagging com Decision Tree: 0.9852375359442712
Precisão do modelo Bagging com Decision Tree: 0.9800849495111396
Recall do modelo Bagging com Decision Tree: 0.9906038637560245
F1-score do modelo Bagging com Decision Tree: 0.9853163333131911
AUC ROC do modelo Bagging com Decision Tree: 0.9852375359442712
Matriz de confusão do modelo Bagging com Decision Tree:
[[24194   497]
 [  232 24459]]


#### Naive Bayes

In [ ]:
# Inicializar o modelo de Naive Bayes com distribuição de Gaussiana
modelo_naive_bayes = GaussianNB()

# Definir o classificador de Bagging com Naive Bayes como base
bagging_clf = BaggingClassifier(estimator=modelo_naive_bayes, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com Naive Bayes:", accuracy_bagging)
print("Precisão do modelo Bagging com Naive Bayes:", precision_bagging)
print("Recall do modelo Bagging com Naive Bayes:", recall_bagging)
print("F1-score do modelo Bagging com Naive Bayes:", f1_score_bagging)
print("AUC ROC do modelo Bagging com Naive Bayes:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com Naive Bayes:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBaggingPosBal.append(accuracy_bagging)
PrecisionBaggingPosBal.append(precision_bagging)
RecallBaggingPosBal.append(recall_bagging)
F1BaggingPosBal.append(f1_score_bagging)
Roc_aucBaggingPosBal.append(auc_roc_bagging)


Acurácia do modelo Bagging com Naive Bayes: 0.7307925964926492
Precisão do modelo Bagging com Naive Bayes: 0.7253707731856832
Recall do modelo Bagging com Naive Bayes: 0.7428212709084282
F1-score do modelo Bagging com Naive Bayes: 0.7339923163118297
AUC ROC do modelo Bagging com Naive Bayes: 0.7307925964926492
Matriz de confusão do modelo Bagging com Naive Bayes:
[[17747  6944]
 [ 6350 18341]]


#### MLP

In [ ]:
# Definir o classificador base (MLP)
modelo_mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=1000)

# Definir o classificador de Bagging com MLP como base
bagging_clf = BaggingClassifier(estimator=modelo_mlp, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com MLP:", accuracy_bagging)
print("Precisão do modelo Bagging com MLP:", precision_bagging)
print("Recall do modelo Bagging com MLP:", recall_bagging)
print("F1-score do modelo Bagging com MLP:", f1_score_bagging)
print("AUC ROC do modelo Bagging com MLP:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com MLP:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBaggingPosBal.append(accuracy_bagging)
PrecisionBaggingPosBal.append(precision_bagging)
RecallBaggingPosBal.append(recall_bagging)
F1BaggingPosBal.append(f1_score_bagging)
Roc_aucBaggingPosBal.append(auc_roc_bagging)


Acurácia do modelo Bagging com MLP: 0.9441496901705074
Precisão do modelo Bagging com MLP: 0.9423581138316325
Recall do modelo Bagging com MLP: 0.9461747195334332
F1-score do modelo Bagging com MLP: 0.944262560122873
AUC ROC do modelo Bagging com MLP: 0.9441496901705075
Matriz de confusão do modelo Bagging com MLP:
[[23262  1429]
 [ 1329 23362]]


#### KNN

In [ ]:
#inicializar o modelo KNN
modelo_knn = KNeighborsClassifier(n_neighbors=7)  # Escolha o número de vizinhos desejado

# Definir o classificador de Bagging com KNN como base
bagging_clf = BaggingClassifier(estimator=modelo_knn, n_estimators=30)

# Treinar o modelo Bagging
bagging_clf.fit(x_train, y_train)

# Prever usando o modelo Bagging
y_pred = bagging_clf.predict(x_test)

# Avaliar a precisão do modelo
accuracy_bagging = accuracy_score(y_test, y_pred)
precision_bagging = precision_score(y_test, y_pred)
recall_bagging = recall_score(y_test, y_pred)
f1_score_bagging = f1_score(y_test, y_pred)
auc_roc_bagging = roc_auc_score(y_test, y_pred)


print("Acurácia do modelo Bagging com KNN:", accuracy_bagging)
print("Precisão do modelo Bagging com KNN:", precision_bagging)
print("Recall do modelo Bagging com KNN:", recall_bagging)
print("F1-score do modelo Bagging com KNN:", f1_score_bagging)
print("AUC ROC do modelo Bagging com KNN:", auc_roc_bagging)

# Matriz de confusão
conf_matrix_bagging = confusion_matrix(y_test, y_pred)
print("Matriz de confusão do modelo Bagging com KNN:")
print(conf_matrix_bagging)

# Adicione a acurácia à lista de acurácias do Bagging
AcurracyBaggingPosBal.append(accuracy_bagging)
PrecisionBaggingPosBal.append(precision_bagging)
RecallBaggingPosBal.append(recall_bagging)
F1BaggingPosBal.append(f1_score_bagging)
Roc_aucBaggingPosBal.append(auc_roc_bagging)


Acurácia do modelo Bagging com KNN: 0.9712648333400834
Precisão do modelo Bagging com KNN: 0.9511476426799007
Recall do modelo Bagging com KNN: 0.9935604066258961
F1-score do modelo Bagging com KNN: 0.9718915278410554
AUC ROC do modelo Bagging com KNN: 0.9712648333400834
Matriz de confusão do modelo Bagging com KNN:
[[23431  1260]
 [  159 24532]]


#### Resultados

In [ ]:
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']
# Criar um DataFrame do Pandas com os valores das variáveis
dfB = pd.DataFrame({'Modelos Bagging': Modelos, 'Acuracia (DREBIN)': AcurracyBaggingPosBal,
                    'Precisão (DREBIN)': PrecisionBaggingPosBal, 'Recall (DREBIN)':RecallBaggingPosBal, 
                    'F1 Score': F1BaggingPosBal, 'AUC ROC': Roc_aucBaggingPosBal})

print(tabulate(dfB, headers='keys', tablefmt='fancy_grid', floatfmt=".4f"))

╒════╤═══════════════════╤═════════════════════╤═════════════════════╤═══════════════════╤════════════╤═══════════╕
│    │ Modelos Bagging   │   Acuracia (DREBIN) │   Precisão (DREBIN) │   Recall (DREBIN) │   F1 Score │   AUC ROC │
╞════╪═══════════════════╪═════════════════════╪═════════════════════╪═══════════════════╪════════════╪═══════════╡
│  0 │ Decision Tree     │              0.9852 │              0.9801 │            0.9906 │     0.9853 │    0.9852 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  1 │ Naive Bayes       │              0.7308 │              0.7254 │            0.7428 │     0.7340 │    0.7308 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼───────────────────┼────────────┼───────────┤
│  2 │ MLP               │              0.9441 │              0.9424 │            0.9462 │     0.9443 │    0.9441 │
├────┼───────────────────┼─────────────────────┼─────────────────────┼──

### Resultados

In [ ]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Acurracy, AcurracyDP)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Precision, PrecisionDP)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Recall, RecallDP)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_score, F1_scoreDP)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_auc, Roc_aucDP)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9817 ± 0.0007,0.7712 ± 0.0105,0.8194 ± 0.0120,0.7945 ± 0.0077,0.9042 ± 0.0059
1,Naive Bayes,0.9163 ± 0.0030,0.2501 ± 0.0098,0.4704 ± 0.0149,0.3264 ± 0.0097,0.7034 ± 0.0070
2,MLP,0.9769 ± 0.0009,0.8171 ± 0.0407,0.6031 ± 0.0410,0.6918 ± 0.0158,0.7984 ± 0.0195
3,KNN,0.9812 ± 0.0007,0.8112 ± 0.0128,0.7347 ± 0.0145,0.7709 ± 0.0095,0.8635 ± 0.0071


In [ ]:
import pandas as pd
array = [0,0,0,0,0]
# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f}" for media, dp in zip(AcurracyBagging, array)],
    'Precisão (DREBIN)': [f"{media:.4f}" for media, dp in zip(PrecisionBagging, array)],
    'Recall (DREBIN)': [f"{media:.4f}" for media, dp in zip(RecallBagging, array)],
    'F1 Score': [f"{media:.4f}" for media, dp in zip(F1Bagging, array)],
    'AUC RoC': [f"{media:.4f}" for media, dp in zip(Roc_aucBagging, array)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9877,0.8994,0.8040,0.8490,0.9000
1,Naive Bayes,0.9178,0.2486,0.4487,0.3200,0.6938
2,MLP,0.9770,0.8281,0.5890,0.6884,0.7918
3,KNN,0.9816,0.8188,0.7356,0.7750,0.8641


In [ ]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(AcurracyPosBal, AcurracyDPPosBal)],
    'Precisão (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(PrecisionPosBal, PrecisionDPPosBal)],
    'Recall (DREBIN)': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(RecallPosBal, RecallDPPosBal)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_scorePosBal, F1_scoreDPPosBal)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_aucPosBal, Roc_aucDPPosBal)]
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9810 ± 0.0006,0.9737 ± 0.0011,0.9887 ± 0.0008,0.9812 ± 0.0006,0.9810 ± 0.0006
1,Naive Bayes,0.7299 ± 0.0046,0.7255 ± 0.0047,0.7397 ± 0.0141,0.7325 ± 0.0067,0.7299 ± 0.0046
2,MLP,0.9348 ± 0.0028,0.9329 ± 0.0085,0.9371 ± 0.0122,0.9349 ± 0.0032,0.9348 ± 0.0028
3,KNN,0.9708 ± 0.0010,0.9517 ± 0.0017,0.9920 ± 0.0014,0.9714 ± 0.0009,0.9708 ± 0.0010


In [ ]:
import pandas as pd
array = [0,0,0,0,0]
# Definir os modelos
Modelos = ['Decision Tree', 'Naive Bayes', 'MLP', 'KNN']

# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia (DREBIN)': [f"{media:.4f}" for media, dp in zip(AcurracyBaggingPosBal, array)],
    'Precisão (DREBIN)': [f"{media:.4f}" for media, dp in zip(PrecisionBaggingPosBal, array)],
    'Recall (DREBIN)': [f"{media:.4f}" for media, dp in zip(RecallBaggingPosBal, array)],
    'F1 Score': [f"{media:.4f}" for media, dp in zip(F1BaggingPosBal, array)],
    'AUC RoC': [f"{media:.4f}" for media, dp in zip(Roc_aucBaggingPosBal, array)]    
})

# Função para destacar o maior valor de cada coluna
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
styled_df = df.style.apply(highlight_max, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df


,Modelos,Acuracia (DREBIN),Precisão (DREBIN),Recall (DREBIN),F1 Score,AUC RoC
0,Decision Tree,0.9852,0.9801,0.9906,0.9853,0.9852
1,Naive Bayes,0.7308,0.7254,0.7428,0.7340,0.7308
2,MLP,0.9441,0.9424,0.9462,0.9443,0.9441
3,KNN,0.9713,0.9511,0.9936,0.9719,0.9713


In [ ]:
import pandas as pd

# Definir os modelos
Modelos = ['Decision Tree DD', 'Naive Bayes DD', 'MLP DD', 'KNN DD',
            'Bagging Decision Tree DD', 'Bagging Naive Bayes DD', 'Bagging MLP DD', 'Bagging KNN DD',
              'Decision Tree DB', 'Naive Bayes DB', 'MLP DB', 'KNN DB', 
              'Bagging Decision Tree DB', 'Bagging Naive Bayes DB', 'Bagging MLP DB', 'Bagging KNN DB']


AcurracyAll = Acurracy + AcurracyBagging + AcurracyPosBal + AcurracyBaggingPosBal
PrecisionAll = Precision + PrecisionBagging + PrecisionPosBal + PrecisionBaggingPosBal
RecallAll = Recall + RecallBagging + RecallPosBal + RecallBaggingPosBal
F1_scoreAll = F1_score + F1Bagging + F1_scorePosBal + F1BaggingPosBal
Roc_aucAll = Roc_auc + Roc_aucBagging + Roc_aucPosBal + Roc_aucBaggingPosBal
AcurracyDPAll = AcurracyDP + [0,0,0,0] + AcurracyDPPosBal + [0,0,0,0]
PrecisionDPAll = PrecisionDP + [0,0,0,0] + PrecisionDPPosBal + [0,0,0,0] 
RecallDPAll = RecallDP + [0,0,0,0] + RecallDPPosBal + [0,0,0,0]
F1_scoreDPAll = F1_scoreDP + [0,0,0,0] + F1_scoreDPPosBal + [0,0,0,0]
Roc_aucDPAll = Roc_aucDP + [0,0,0,0] + Roc_aucDPPosBal + [0,0,0,0]

# print(len(AcurracyAll), len(PrecisionAll), len(RecallAll), len(F1_scoreAll), len(Roc_aucAll), len(AcurracyDPAll), 
#       len(PrecisionDPAll), len(RecallDPAll), len(F1_scoreDPAll), len(Roc_aucDPAll))


# Criar um DataFrame do Pandas com os valores das variáveis
df = pd.DataFrame({
    'Modelos': Modelos,
    'Acuracia': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(AcurracyAll, AcurracyDPAll)],
    'Precisão': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(PrecisionAll, PrecisionDPAll)],
    'Recall': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(RecallAll, RecallDPAll)],
    'F1 Score': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(F1_scoreAll, F1_scoreDPAll)],
    'AUC RoC': [f"{media:.4f} ± {dp:.4f}" for media, dp in zip(Roc_aucAll, Roc_aucDPAll)]
})

# Aplicar a função de destaque apenas para as colunas numéricas
numeric_columns = df.columns[1:]
highlight_colors = ['green', 'blue', 'red']
def highlight_top_n_with_colors(s, n=3, colors=highlight_colors):
    sorted_index = np.argsort(s.values)[::-1]
    top_n_index = sorted_index[:n]    
    return [f'background-color: {colors[np.where(top_n_index == index)[0][0]]}' if index in top_n_index else '' for index in s.index]

styled_df = df.style.apply(highlight_top_n_with_colors, subset=numeric_columns)

# Definir a cor do texto para a tabela
styled_df.set_properties(**{'color': 'white'}, subset=pd.IndexSlice[:, :])

# Exibir o DataFrame estilizado no Jupyter Notebook
styled_df

,Modelos,Acuracia,Precisão,Recall,F1 Score,AUC RoC
0,Decision Tree DD,0.9817 ± 0.0007,0.7712 ± 0.0105,0.8194 ± 0.0120,0.7945 ± 0.0077,0.9042 ± 0.0059
1,Naive Bayes DD,0.9163 ± 0.0030,0.2501 ± 0.0098,0.4704 ± 0.0149,0.3264 ± 0.0097,0.7034 ± 0.0070
2,MLP DD,0.9769 ± 0.0009,0.8171 ± 0.0407,0.6031 ± 0.0410,0.6918 ± 0.0158,0.7984 ± 0.0195
3,KNN DD,0.9812 ± 0.0007,0.8112 ± 0.0128,0.7347 ± 0.0145,0.7709 ± 0.0095,0.8635 ± 0.0071
4,Bagging Decision Tree DD,0.9877 ± 0.0000,0.8994 ± 0.0000,0.8040 ± 0.0000,0.8490 ± 0.0000,0.9000 ± 0.0000
5,Bagging Naive Bayes DD,0.9178 ± 0.0000,0.2486 ± 0.0000,0.4487 ± 0.0000,0.3200 ± 0.0000,0.6938 ± 0.0000
6,Bagging MLP DD,0.9770 ± 0.0000,0.8281 ± 0.0000,0.5890 ± 0.0000,0.6884 ± 0.0000,0.7918 ± 0.0000
7,Bagging KNN DD,0.9816 ± 0.0000,0.8188 ± 0.0000,0.7356 ± 0.0000,0.7750 ± 0.0000,0.8641 ± 0.0000
8,Decision Tree DB,0.9810 ± 0.0006,0.9737 ± 0.0011,0.9887 ± 0.0008,0.9812 ± 0.0006,0.9810 ± 0.0006
9,Naive Bayes DB,0.7299 ± 0.0046,0.7255 ± 0.0047,0.7397 ± 0.0141,0.7325 ± 0.0067,0.7299 ± 0.0046
